# Imports

In [6]:
# Import modules
import logging
import threading
import time
import xlwings as xw

# Constants

In [7]:
# Excel font constants
HEADINGS_FONT = "Lato"
TABLE_FONT = "Source Sans Pro"
LARGE_FONT = 18
SMALL_FONT = 11

# First row
FIRST_COLUMN_WIDTH = 0.5
FIRST_ROW_HEIGHT = 4.5

# Heading and table
HEADINGS_ROW_HEIGHT = 24
TABLE_ROW_HEIGHT = 18

# Excel text alignment constants
XL_H_ALIGN_CENTER = -4108  # xlHAlignCenter
XL_V_ALIGN_CENTER = -4108  # xlVAlignCenter
XL_V_ALIGN_TOP = -4160  # xlVAlignTop

# Excel colour constants
MAPEI_GREY = 15132135  # RGB (231, 229, 230)
FULL_WHITE = 16777215  # RGB(255, 255, 255)

# Functions

In [8]:
def start_log():
    """
    Initializes the logging configuration for the application.
    """
    logging.basicConfig(
        filename='App.log',
        filemode='w',
        format='%(message)s',
        level=logging.DEBUG,
)


def end_log():
    """
    Closes the logging configuration for the application.
    """
    # Fetch the root logger
    logger = logging.getLogger()
    
    # Close all handlers of the logger
    for handler in logger.handlers[:]:
        handler.close()
        logger.removeHandler(handler)

# Classes

In [9]:
class TestConnection:
    """
    Checks a connection can be established with Excel.
    """
    def __init__(self):
        """
        Initializes a connection to Excel, ensuring it's open and not in data entry mode.
        """
        # Establish an Excel connection
        self.xlApp = xw.apps.active
        if self.xlApp is None:
            raise Exception("Excel must be open to continue!")
        
        # Test if the workbook is accessible (not in data entry mode)
        self.workbook_connection_tested = False
        t1 = threading.Thread(target=self._test_connection)
        t2 = threading.Thread(target=self._connection_timer)
        t1.start(), t2.start()
        t1.join(), t2.join()
        self.wb = xw.books.active


    def _test_connection(self):
        """
        A thread to try and connect to Excel.
        """
        wb = xw.books.active
        wb = None
        self.workbook_connection_tested = True
        
    def _connection_timer(self):
        """
        A timer thread to identify if Excel is in data entry mode.
        """
        time.sleep(0.1)
        if not self.workbook_connection_tested:
            print("Excel is in data entry mode! Select Excel and press 'esc' key to continue.")


class ExcelWorksheet():
    
    def __init__(self, xlApp, wb,search_term_columns=3):
        """
        Initializes the ImportData class.
        
        Parameters:
            xlApp (object): The Excel application object.
            wb (object): The Excel workbook
        """
        self.xlApp = xlApp
        self.wb = wb
        self.search_term_columns = search_term_columns

        logging.info(f"self.xlApp = {self.xlApp}")
        logging.info(f"self.wb = {self.wb.fullname}")
        logging.info(f"Number of search columns = {self.search_term_columns}")
    
    def make_sheet(self):
        """
        Make a search terms worksheet in excel.
        """
        self._connect_to_worksheet()
        self._set_range_variables()
        self._headings()
        self._format_worksheet()

    def _connect_to_worksheet(self):  
        """
        Select requested worksheet or make a new worksheet.
        """
        try:
            ws = self.wb.sheets[0]
        except:
            ws = self.wb.sheets.add(name=None, before=self.wb.sheets[0])
            
        # If selected worksheet has existing data make a new worksheet
        data_range = ws.used_range.address.replace("$", "").split(":")
        if len(data_range) > 1: 
            ws = self.wb.sheets.add(name=None, before=self.wb.sheets[0])
        try:
            ws.name = "search terms"
        except Exception:
            print("Can't name worksheet 'search terms' as name already in use.")
        finally:
            self.ws = ws # Return the worksheet object    
    
    def _set_range_variables(self):

        end_col_letter = chr(ord('B') + self.search_term_columns * 2 - 1)
        penultimate_col_letter = chr(ord('B') + self.search_term_columns * 2 - 2)

        self.headings_data = self.ws.range(f"B2:{end_col_letter}2")
        self.headings_range = self.ws.range(f"B2:{penultimate_col_letter}2")
        self.table_data = self.ws.range(f"B3:{end_col_letter}100")

    def _headings(self):
        """
        Generate and set column headings.
        """
        # Dynamically create headings based on the number of search columns
        headings = [f"Search Terms {i+1}" for i in range(self.search_term_columns)]
        widths = [39.43 for _ in range(self.search_term_columns)]

        # Insert blank-named spacing columns
        for idx in range(len(headings), -1, -1):
            headings.insert(idx, "")
            widths.insert(idx, 0.5)

        # Add column headings and widths to the worksheet
        for idx in range(len(headings)):
            col_idx = idx + 1
            self.ws.range(2, col_idx).value = headings[idx]
            self.ws.range(2, col_idx).column_width = widths[idx]
            
    def _format_worksheet(self):
        """
        Format the worksheet.
        """
        # Format the first row and column
        self.ws.range("A:A").column_width = FIRST_COLUMN_WIDTH
        self.ws.range("1:1").row_height = FIRST_ROW_HEIGHT

        # Format the title bar
        self.headings_range.api.Interior.Color = MAPEI_GREY

        # Format the headings text
        self.headings_data.font.name = HEADINGS_FONT
        self.headings_data.font.size = LARGE_FONT
        self.headings_data.api.HorizontalAlignment = XL_H_ALIGN_CENTER
        self.headings_data.api.VerticalAlignment = XL_V_ALIGN_CENTER
        self.headings_data.row_height = HEADINGS_ROW_HEIGHT
        self.headings_data.font.color = FULL_WHITE
        self.headings_data.font.bold = True

        # Format the table text
        self.table_data.font.name = TABLE_FONT
        self.table_data.font.size = SMALL_FONT
        self.table_data.api.HorizontalAlignment = XL_H_ALIGN_CENTER
        self.table_data.api.VerticalAlignment = XL_V_ALIGN_CENTER
        self.table_data.row_height = TABLE_ROW_HEIGHT
        self.table_data.wrap_text = True
        
        # Freeze titlebar and headings rows
        self.ws.range("3:3").select()
        self.xlApp.api.ActiveWindow.FreezePanes = True
        
        # Select top table cell
        self.ws.range("B3").select()
       

# Main

In [10]:
def make_search_terms_worksheet(**kwags):
    """
    Orchestrator function that controls the overall execution of the application.

    Steps:
    1. Tests the connection to an open excel worksheet.
    2. Creates a excel search terms worksheet
    """
    # Run a test to check connection with Excel application. 
    test_connection = TestConnection()
    xlApp = test_connection.xlApp
    wb = test_connection.wb

    # Make search terms worksheet.
    logging.info(f"Running ExcelWorksheet Class  -------------")
    excel_worksheet = ExcelWorksheet(xlApp, wb, kwags['number_of_columns'])
    excel_worksheet.make_sheet()
    logging.info(f"-----------------------------------------\n\n")

    # Confirmation statement
    print("Task Complete...")

    
def main():    
    # Enable or disable logging (comment/uncomment the next line) 
    start_log()
    
    # Call make search_terms_worksheet function (select number columns between 2 and 5)
    make_search_terms_worksheet(number_of_columns=3)

    # End logging and close file
    end_log()

if __name__ == "__main__":
    main()

Task Complete...
